----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Rainfall Prediction Model
##Traning Set Accuracy: 0.9907
##Test Set Accuracy: 0.9975

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf
import os
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
df1 = pd.read_csv( os.path.join('dataset', 'combined_data.csv') )
df1.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [4]:


df1['Date'] = pd.Categorical(df1['Date'])
df1['Date'] = df1.Date.cat.codes

df1['Location'] = pd.Categorical(df1['Location'])
df1['Location'] = df1.Location.cat.codes

df1['WindGustDir'] = pd.Categorical(df1['WindGustDir'])
df1['WindGustDir'] = df1.WindGustDir.cat.codes

df1['WindDir9am'] = pd.Categorical(df1['WindDir9am'])
df1['WindDir9am'] = df1.WindDir9am.cat.codes

df1['WindDir3pm'] = pd.Categorical(df1['WindDir3pm'])
df1['WindDir3pm'] = df1.WindDir3pm.cat.codes

df1['RainToday'] = pd.Categorical(df1['RainToday'])
df1['RainToday'] = df1.RainToday.cat.codes

df1['RainTomorrow'] = pd.Categorical(df1['RainTomorrow'])
df1['RainTomorrow'] = df1.RainTomorrow.cat.codes

#df1.interpolate(method = 'linear')

df1.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,396,2,13.4,22.9,0.6,NaN,NaN,13,44.0,13,14,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0,0.0,0
1,397,2,7.4,25.1,0.0,NaN,NaN,14,44.0,6,15,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0,0.0,0
2,398,2,12.9,25.7,0.0,NaN,NaN,15,46.0,13,15,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0,0.0,0
3,399,2,9.2,28.0,0.0,NaN,NaN,4,24.0,9,0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0,1.0,0
4,400,2,17.5,32.3,1.0,NaN,NaN,13,41.0,1,7,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0.2,0


In [5]:
df1.count()

Date             142193
Location         142193
MinTemp          141556
MaxTemp          141871
Rainfall         140787
Evaporation       81350
Sunshine          74377
WindGustDir      142193
WindGustSpeed    132923
WindDir9am       142193
WindDir3pm       142193
WindSpeed9am     140845
WindSpeed3pm     139563
Humidity9am      140419
Humidity3pm      138583
Pressure9am      128179
Pressure3pm      128212
Cloud9am          88536
Cloud3pm          85099
Temp9am          141289
Temp3pm          139467
RainToday        142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64

In [0]:
df1 = df1.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Pressure9am', 'Pressure3pm'], axis=1)

In [7]:
df1.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,396,2,13.4,22.9,0.6,13,44.0,13,14,20.0,24.0,71.0,22.0,16.9,21.8,0,0.0,0
1,397,2,7.4,25.1,0.0,14,44.0,6,15,4.0,22.0,44.0,25.0,17.2,24.3,0,0.0,0
2,398,2,12.9,25.7,0.0,15,46.0,13,15,19.0,26.0,38.0,30.0,21.0,23.2,0,0.0,0
3,399,2,9.2,28.0,0.0,4,24.0,9,0,11.0,9.0,45.0,16.0,18.1,26.5,0,1.0,0
4,400,2,17.5,32.3,1.0,13,41.0,1,7,7.0,20.0,82.0,33.0,17.8,29.7,0,0.2,0


##Perform Interpolation to handle NaN Values in the Data

In [0]:
df1 = df1.interpolate(method = 'linear')

In [9]:
df1.count()

Date             142193
Location         142193
MinTemp          142193
MaxTemp          142193
Rainfall         142193
WindGustDir      142193
WindGustSpeed    142193
WindDir9am       142193
WindDir3pm       142193
WindSpeed9am     142193
WindSpeed3pm     142193
Humidity9am      142193
Humidity3pm      142193
Temp9am          142193
Temp3pm          142193
RainToday        142193
RISK_MM          142193
RainTomorrow     142193
dtype: int64

In [10]:
df1.dtypes

Date               int16
Location            int8
MinTemp          float64
MaxTemp          float64
Rainfall         float64
WindGustDir         int8
WindGustSpeed    float64
WindDir9am          int8
WindDir3pm          int8
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Temp9am          float64
Temp3pm          float64
RainToday           int8
RISK_MM          float64
RainTomorrow        int8
dtype: object

##Split the dataset into train and test dataset 
###Pop the RainTomorrow attribute and use it as the label for the train and test data

In [11]:
train1, test1 = train_test_split(df1, test_size=0.2)
print(len(train1), 'train examples')
print(len(test1), 'test examples')

train_target1 = train1.pop('RainTomorrow')
test_target1 = test1.pop('RainTomorrow')

113754 train examples
28439 test examples


In [0]:
train_dataset1 = tf.data.Dataset.from_tensor_slices((train1.values, train_target1.values))

In [0]:
train_dataset1 = train_dataset1.shuffle(len(train1)).batch(8)

##Defining the NN Configuration

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(9, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adamax',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [20]:
model1 = get_compiled_model()
model1.fit(train_dataset1, epochs=10)

Epoch 1/10
14220/14220 [==============================] - 29s 2ms/step - loss: 10.7653 - accuracy: 0.2253
Epoch 2/10
14220/14220 [==============================] - 28s 2ms/step - loss: 0.1606 - accuracy: 0.9173
Epoch 3/10
14220/14220 [==============================] - 28s 2ms/step - loss: 0.0567 - accuracy: 0.9786
Epoch 4/10
14220/14220 [==============================] - 28s 2ms/step - loss: 0.0370 - accuracy: 0.9863
Epoch 5/10
14220/14220 [==============================] - 27s 2ms/step - loss: 0.0318 - accuracy: 0.9879
Epoch 6/10
14220/14220 [==============================] - 27s 2ms/step - loss: 0.0290 - accuracy: 0.9885
Epoch 7/10
14220/14220 [==============================] - 28s 2ms/step - loss: 0.0274 - accuracy: 0.9895
Epoch 8/10
14220/14220 [==============================] - 27s 2ms/step - loss: 0.0250 - accuracy: 0.9907
Epoch 9/10
14220/14220 [==============================] - 27s 2ms/step - loss: 0.0245 - accuracy: 0.9905
Epoch 10/10
14220/14220 [=============================

In [21]:
test_dataset1 = tf.data.Dataset.from_tensor_slices((test1.values, test_target1.values))
test_dataset1 = test_dataset1.shuffle(len(test1)).batch(8)
test_loss1, test_accuracy1 = model1.evaluate(test_dataset1)
print("test loss:",test_loss1, "test accuracy:",test_accuracy1)

3555/3555 [==============================] - 4s 1ms/step - loss: 0.0104 - accuracy: 0.9975
test loss: 0.010411802379978087 test accuracy: 0.9975034
